In [ ]:
# Instalar rpy2 para usar R
!pip install -q rpy2

# Activar celda mágica para R
%load_ext rpy2.ipython

# Subir archivo
from google.colab import files
uploaded = files.upload()

# Cargar y explorar
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Leer archivo
df = pd.read_csv("spotify_history.csv")

# Mostrar columnas
print("Columnas:", df.columns.tolist())

# Eliminar duplicados
df = df.drop_duplicates()

# Convertir fechas
df['ts'] = pd.to_datetime(df['ts'])
df['hour'] = df['ts'].dt.hour
df['day_of_week'] = df['ts'].dt.day_name()

# Convertir duración a minutos
df['minutes_played'] = df['ms_played'] / 60000

# Estadísticas generales
print("\nNúmero total de registros:", len(df))
print("\nDuración total reproducida (horas):", round(df['minutes_played'].sum() / 60, 2))
print("\nPromedio de minutos por canción:", round(df['minutes_played'].mean(), 2))

# Canciones más reproducidas
top_tracks = df['track_name'].value_counts().head(10)
print("\nTop 10 canciones más reproducidas:")
print(top_tracks)

# Artistas más reproducidos
top_artists = df['artist_name'].value_counts().head(10)
print("\nTop 10 artistas más reproducidos:")
print(top_artists)

# Días con más actividad
day_activity = df['day_of_week'].value_counts()
print("\nActividad por día de la semana:")
print(day_activity)

# Horas con más actividad
hour_activity = df['hour'].value_counts().sort_index()
print("\nActividad por hora del día:")
print(hour_activity)

# 🔹 Visualización: actividad por día
plt.figure(figsize=(8,5))
sns.countplot(data=df, x='day_of_week', order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.title('Reproducciones por Día de la Semana')
plt.ylabel('Número de canciones')
plt.xlabel('Día')
plt.xticks(rotation=45)
plt.show()

# 🔹 Visualización: actividad por hora
plt.figure(figsize=(10,5))
sns.histplot(df['hour'], bins=24, kde=False)
plt.title('Reproducciones por Hora del Día')
plt.xlabel('Hora')
plt.ylabel('Número de canciones')
plt.show()

# 🔹 Visualización: Heatmap día vs hora
pivot_table = df.pivot_table(index='day_of_week', columns='hour', values='minutes_played', aggfunc='sum')
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
pivot_table = pivot_table.reindex(ordered_days)

plt.figure(figsize=(12,6))
sns.heatmap(pivot_table, cmap='YlGnBu')
plt.title("Heatmap: Actividad de Escucha por Día y Hora")
plt.xlabel("Hora del día")
plt.ylabel("Día de la semana")
plt.show()

# Transferencia a R
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()
ro.globalenv['spotify_df'] = pandas2ri.py2rpy(df)


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


KeyboardInterrupt: 

In [ ]:
# Visualizaciones enriquecidas usando seaborn y matplotlib

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Estilo visual general
sns.set(style="whitegrid")

# 1. Distribución de duración de las canciones reproducidas
plt.figure(figsize=(8,5))
sns.histplot(df['minutes_played'], bins=50, kde=True, color='mediumseagreen')
plt.title('Distribución de Minutos Reproducidos por Canción')
plt.xlabel('Minutos reproducidos')
plt.ylabel('Frecuencia')
plt.show()

# 2. Top 10 artistas con más tiempo de reproducción (no solo por frecuencia)
top_artists_time = df.groupby('artist_name')['minutes_played'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,6))
sns.barplot(x=top_artists_time.values, y=top_artists_time.index, palette='Blues_d')
plt.title('Top 10 Artistas con Más Tiempo de Reproducción')
plt.xlabel('Minutos reproducidos')
plt.ylabel('Artista')
plt.show()

# 3. Boxplot: minutos por día de la semana (ver distribución y outliers)
plt.figure(figsize=(10,6))
sns.boxplot(data=df, x='day_of_week', y='minutes_played',
            order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
            palette='pastel')
plt.title('Distribución de Minutos por Día de la Semana')
plt.xlabel('Día')
plt.ylabel('Minutos reproducidos por canción')
plt.ylim(0, 10)  # Limitar para mejor visualización
plt.show()

# 4. Relación entre hora del día y duración de reproducción
plt.figure(figsize=(10,6))
sns.scatterplot(data=df, x='hour', y='minutes_played', alpha=0.3, color='purple')
plt.title('Relación entre Hora del Día y Duración de Reproducción')
plt.xlabel('Hora')
plt.ylabel('Minutos reproducidos')
plt.xticks(range(0,24))
plt.show()

# 5. Canciones más reproducidas (top 10 por frecuencia)
top_tracks = df['track_name'].value_counts().head(10)

plt.figure(figsize=(10,6))
sns.barplot(x=top_tracks.values, y=top_tracks.index, palette='mako')
plt.title('Top 10 Canciones Más Reproducidas')
plt.xlabel('Número de veces reproducida')
plt.ylabel('Canción')
plt.show()

# 6. Comparación: razón de inicio de la canción
plt.figure(figsize=(8,5))
sns.countplot(data=df, y='reason_start', order=df['reason_start'].value_counts().index, palette='Set2')
plt.title('Razón por la que Inició la Canción')
plt.xlabel('Número de canciones')
plt.ylabel('Razón de inicio')
plt.show()

# 7. Comparación: plataforma de reproducción
plt.figure(figsize=(8,5))
sns.countplot(data=df, y='platform', order=df['platform'].value_counts().index, palette='rocket')
plt.title('Plataforma de Reproducción')
plt.xlabel('Número de canciones')
plt.ylabel('Plataforma')
plt.show()

# 8. Gráfico combinado día vs duración total
total_minutes_by_day = df.groupby('day_of_week')['minutes_played'].sum().reindex(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

plt.figure(figsize=(10,6))
sns.barplot(x=total_minutes_by_day.index, y=total_minutes_by_day.values, palette='coolwarm')
plt.title('Duración Total de Reproducción por Día')
plt.ylabel('Minutos')
plt.xlabel('Día de la Semana')
plt.show()


In [ ]:
%%R
# Instalar librerías si es necesario
if (!require('dplyr')) install.packages('dplyr', repos='http://cran.us.r-project.org')
if (!require('ggplot2')) install.packages('ggplot2', repos='http://cran.us.r-project.org')

# Cargar librerías
library(dplyr)
library(ggplot2)

# Asegurar que sea un data.frame
spotify_df <- as.data.frame(spotify_df)

# Top 10 artistas más escuchados
top_artists <- spotify_df %>%
  group_by(artist_name) %>%
  summarise(total_plays = n()) %>%
  arrange(desc(total_plays)) %>%
  head(10)

print(top_artists)

# Gráfico de barras
ggplot(top_artists, aes(x = reorder(artist_name, total_plays), y = total_plays)) +
  geom_bar(stat = 'identity', fill = 'steelblue') +
  coord_flip() +
  labs(title = 'Top 10 Artistas Más Escuchados', x = 'Artista', y = 'Reproducciones')
